# PyCitySchools - Pandas Assignment

## Written Analysis
Maths and reading scores from multiple schools in a local governement area were analysed to gain insight into potential factors that might influence academic scores. 

It was found that students fared slightly better in maths than reading, and that independent schools, on average, had higher average maths and reading scores than government schools, which translated into more students passing the respective subject. Similarly, students from smaller schools had higher marks across the board compared to students from larger schools. However, the former and latter points are linked, as independent schools are often smaller than government schools. This phenomenon might be explained by a greater teacher to student ratio for independent schools, but teacher number and/or class size data was lacking from this analysis and therefore this hypothesis could not be tested.

-- Samuel Palframan, June 2023.

In [11]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [12]:
#Number of unique schools
unique_schools = school_data_complete["school_name"].unique()
no_unique_schools = len(unique_schools)

#Total number of students
no_students = len(school_data_complete["student_name"])

#Total budget
total_budget = school_data["budget"].sum()

#Average maths score
maths_average = student_data["maths_score"].mean()

#Average reading score
reading_average = student_data["reading_score"].mean()

#Percentage of students that passed maths
percentage_passed_maths = (((student_data["maths_score"] >= 50).sum() / no_students)*100)

#Percentage of students that passed reading
percentage_passed_reading = (((student_data["reading_score"] >= 50).sum() / no_students)*100)

#Percentage of students that passed both maths & reading
no_passed_both = len(student_data[(student_data["maths_score"] >= 50) & (student_data["reading_score"] >= 50)])
percentage_passed_both = ((no_passed_both / no_students)*100)

#Summary table
area_summary = pd.DataFrame({"Total Schools": [no_unique_schools],
                              "Total Students": no_students,
                              "Total Budget": total_budget,
                              "Average Maths Score": maths_average,
                              "Average Reading Score": reading_average,
                              "% Passing Maths": percentage_passed_maths,
                              "% Passing Reading": percentage_passed_reading,
                              "% Overall Passing": percentage_passed_both})

#Convert Total Budget to currency
area_summary["Total Budget"] = area_summary["Total Budget"].astype(float).map("${:,.2f}".format)

#Display Summary table
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [13]:
#Constructing Summary Table for Schools

#Set school as index, get school type
school_types = school_data.set_index(["school_name"])["type"]

#Group school and total students
school_student_no = school_data.groupby(["school_name"])['size'].mean().astype(int)

#Group school and budget
school_budget = school_data.groupby(["school_name"])['budget'].mean()#.map("${:,.2f}".format)

#School budget per student
student_budget = (school_budget / school_student_no)#.astype(float).map("${:,.2f}".format)

#Group school name and mean maths score
per_school_maths = school_data_complete.groupby(["school_name"])["maths_score"].mean().round(2)

#Group school name and mean reading score
per_school_reading = school_data_complete.groupby(["school_name"])['reading_score'].mean().round(2)

#Find students that passed maths
pass_maths = school_data_complete.loc[school_data_complete["maths_score"] >=50]
#Count how many students passed maths by school
no_pass_maths = pass_maths.groupby("school_name").size()
#Count total number of students by school
school_students = school_data_complete.groupby('school_name').size()
#Calculate percentage of students that passed maths
percentage_maths = ((no_pass_maths / school_students) * 100).round(2)

#Find students that passed reading
pass_reading = school_data_complete.loc[school_data_complete["reading_score"] >=50]
#Count how many students passed reading by school
no_pass_reading = pass_reading.groupby("school_name").size()
#Count total number of students by school
school_students = school_data_complete.groupby('school_name').size()
#Calculate percentage of students that passed reading
percentage_reading = ((no_pass_reading / school_students) * 100).round(2)

#Find students that passed both maths and reading
passed_both = school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)]

no_passed_both = passed_both.groupby('school_name').size()
percentage_both = ((no_passed_both / school_students) * 100).round(2)
#percentage_both

#Create Summary DataFrame
per_school_summary = pd.DataFrame({"School Type": school_types,
                                     "Total Students": school_student_no,
                                     "Total School Budget": school_budget,
                                     "Per Student Budget": student_budget,
                                     "Average Maths Score": per_school_maths,
                                     "Average Reading Score": per_school_reading,
                                     "% Passing Maths": percentage_maths,
                                     "% Passing Reading": percentage_reading,
                                     "% Overall Passing": percentage_both})

#Convert School Total Budget to currency
#per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].astype(float).map("${:,.2f}".format)

per_school_summary

###GENERAL NOTES: finding students that passed math can be done in 2 lines, see below###
##per_pass_maths = school_data_complete.loc[school_data_complete["maths_score"] >=50]
##((per_pass_maths.groupby("school_name").size() / school_data_complete.groupby('school_name').size())*100).round(2)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,3124928.0,628.0,72.35,71.01,91.64,87.38,80.08
Cabrera High School,Independent,1858,1081356.0,582.0,71.66,71.36,90.85,89.07,80.79
Figueroa High School,Government,2949,1884411.0,639.0,68.70,69.08,81.65,82.81,67.65
Ford High School,Government,2739,1763916.0,644.0,69.09,69.57,82.44,82.22,67.47
Griffin High School,Independent,1468,917500.0,625.0,71.79,71.25,91.21,88.49,81.34
Hernandez High School,Government,4635,3022020.0,652.0,68.87,69.19,80.95,81.88,66.36
Holden High School,Independent,427,248087.0,581.0,72.58,71.66,89.93,88.52,78.92
Huang High School,Government,2917,1910635.0,655.0,68.94,68.91,81.69,81.45,66.71
Johnson High School,Government,4761,3094650.0,650.0,68.84,69.04,82.06,81.98,67.19


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [14]:
#Using sort_value() function to sort Schools based on % Overall Passing score (Top Performing)
top_performing_df = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_performing_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,917500.0,625.0,71.79,71.25,91.21,88.49,81.34
Cabrera High School,Independent,1858,1081356.0,582.0,71.66,71.36,90.85,89.07,80.79
Bailey High School,Government,4976,3124928.0,628.0,72.35,71.01,91.64,87.38,80.08
Wright High School,Independent,1800,1049400.0,583.0,72.05,70.97,91.78,86.67,79.72
Rodriguez High School,Government,3999,2547363.0,637.0,72.05,70.94,90.80,87.40,79.42


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [15]:
#Using sort_value() function to sort Schools based on % Overall Passing score (Bottom Performing)
bottom_performing_df = per_school_summary.sort_values("% Overall Passing")
bottom_performing_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,3022020.0,652.0,68.87,69.19,80.95,81.88,66.36
Huang High School,Government,2917,1910635.0,655.0,68.94,68.91,81.69,81.45,66.71
Johnson High School,Government,4761,3094650.0,650.0,68.84,69.04,82.06,81.98,67.19
Wilson High School,Independent,2283,1319574.0,578.0,69.17,68.88,82.79,81.30,67.46
Ford High School,Government,2739,1763916.0,644.0,69.09,69.57,82.44,82.22,67.47


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [16]:
###YEAR 9###
#Filter DataFrame using .loc for required data
year_9 = school_data_complete.loc[school_data_complete['year'] == 9]
#Group school name and maths score
year_9_maths = year_9.groupby(["school_name"])['maths_score'].mean().round(2)

###YEAR 10###
#Filter DataFrame using .loc for required data
year_10 = school_data_complete.loc[school_data_complete['year'] == 10]
#Group school name and maths score
year_10_maths = year_10.groupby(["school_name"])['maths_score'].mean().round(2)

###YEAR 11###
#Filter DataFrame using .loc for required data
year_11 = school_data_complete.loc[school_data_complete['year'] == 11]
#Group school name and maths score
year_11_maths = year_11.groupby(["school_name"])['maths_score'].mean().round(2)

###YEAR 12###
#Filter DataFrame using .loc for required data
year_12 = school_data_complete.loc[school_data_complete['year'] == 12]
#Group school name and maths score
year_12_maths = year_12.groupby(["school_name"])['maths_score'].mean().round(2)

#Create Summary Maths Score Table by Year
maths_scores_by_year = pd.DataFrame({"Year 9": year_9_maths,
                                     "Year 10": year_10_maths,
                                     "Year 11": year_11_maths,
                                     "Year 12": year_12_maths})

maths_scores_by_year



###GENERAL NOTES###
#First Filter by 'year' using .loc (this keeps the dataframe intact, but just filters by year).
#Then use groupby after filtering to group together the desired columns.

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.49,71.90,72.37,72.68
Cabrera High School,72.32,72.44,71.01,70.60
Figueroa High School,68.48,68.33,68.81,69.33
Ford High School,69.02,69.39,69.25,68.62
Griffin High School,72.79,71.09,71.69,71.47
Hernandez High School,68.59,68.87,69.15,68.99
Holden High School,70.54,75.11,71.64,73.41
Huang High School,69.08,68.53,69.43,68.64
Johnson High School,69.47,67.99,68.64,69.29


## Reading Score by Year

* Perform the same operations as above for reading scores

In [17]:
###YEAR 9###
#Filter DataFrame using .loc for required data
year_9 = school_data_complete.loc[school_data_complete['year'] == 9]
#Group school name and reading score
year_9_reading = year_9.groupby(["school_name"])['reading_score'].mean().round(2)

###YEAR 10###
#Filter DataFrame using .loc for required data
year_10 = school_data_complete.loc[school_data_complete['year'] == 10]
#Group school name and reading score
year_10_reading = year_10.groupby(["school_name"])['reading_score'].mean().round(2)

###YEAR 11###
#Filter DataFrame using .loc for required data
year_11 = school_data_complete.loc[school_data_complete['year'] == 11]
#Group school name and reading score
year_11_reading = year_11.groupby(["school_name"])['reading_score'].mean().round(2)

###YEAR 12###
#Filter DataFrame using .loc for required data
year_12 = school_data_complete.loc[school_data_complete['year'] == 12]
#Group school name and reading score
year_12_reading = year_12.groupby(["school_name"])['reading_score'].mean().round(2)

#Create Summary Reading Score Table by Year
reading_scores_by_year = pd.DataFrame({"Year 9": year_9_reading,
                                       "Year 10": year_10_reading,
                                       "Year 11": year_11_reading,
                                       "Year 12": year_12_reading})

reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.90,70.85,70.32,72.20
Cabrera High School,71.17,71.33,71.20,71.86
Figueroa High School,70.26,67.68,69.15,69.08
Ford High School,69.62,68.99,70.74,68.85
Griffin High School,72.03,70.75,72.39,69.43
Hernandez High School,68.48,70.62,68.42,69.24
Holden High School,71.60,71.10,73.31,70.48
Huang High School,68.67,69.52,68.74,68.67
Johnson High School,68.72,69.30,69.97,67.99


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
#Create bins and corresponding labels
spending_bins = [0, 584, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#Slice data and put into bins
pd.cut(per_school_summary["Per Student Budget"], spending_bins, labels=spending_labels).head()

#Create new column in Summary Table to store binned data
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"], spending_bins, labels=spending_labels)

#Set new column as index
summary_reindexed = per_school_summary.set_index('Spending Ranges (Per Student)')

#Groupby Spending Ranges and relevant columns
sr_maths = summary_reindexed.groupby(['Spending Ranges (Per Student)'])['Average Maths Score'].mean().round(2)
sr_reading = summary_reindexed.groupby(['Spending Ranges (Per Student)'])['Average Reading Score'].mean().round(2)
sr_p_maths = summary_reindexed.groupby(['Spending Ranges (Per Student)'])['% Passing Maths'].mean().round(2)
sr_p_reading = summary_reindexed.groupby(['Spending Ranges (Per Student)'])['% Passing Reading'].mean().round(2)
sr_p_overall = summary_reindexed.groupby(['Spending Ranges (Per Student)'])['% Overall Passing'].mean().round(2)

#Create new DataFrame for above data
spending_summary = pd.DataFrame({"Average Maths Score": sr_maths,
                                 "Average Reading Score": sr_reading,
                                 "% Passing Maths": sr_p_maths,
                                 "% Passing Reading": sr_p_reading,
                                 "% Overall Passing": sr_p_overall})
spending_summary



,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.06,71.03,91.52,87.29,79.88
$630-645,69.86,69.84,84.68,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.75


## Scores by School Size

* Perform the same operations as above, based on school size.

In [19]:
#Create bins and corresponding labels
school_bins = [0, 999, 2000, 5000]
school_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Slice data and put into bins
pd.cut(per_school_summary["Total Students"], school_bins, labels=school_labels).head()

#Create new column in Summary Table to store binned data
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], school_bins, labels=school_labels)

#Set new column as index
school_s_reindexed = per_school_summary.set_index('School Size')

#Groupby School Size and relevant columns
ss_maths = school_s_reindexed.groupby(['School Size'])['Average Maths Score'].mean().round(2)
ss_reading = school_s_reindexed.groupby(['School Size'])['Average Reading Score'].mean().round(2)
ss_p_maths = school_s_reindexed.groupby(['School Size'])['% Passing Maths'].mean().round(2)
ss_p_reading = school_s_reindexed.groupby(['School Size'])['% Passing Reading'].mean().round(2)
ss_p_overall = school_s_reindexed.groupby(['School Size'])['% Overall Passing'].mean().round(2)

#Create new DataFrame for above data
size_summary = pd.DataFrame({"Average Maths Score": ss_maths,
                            "Average Reading Score": ss_reading,
                            "% Passing Maths": ss_p_maths,
                            "% Passing Reading": ss_p_reading,
                            "% Overall Passing": ss_p_overall})
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.63,90.80,87.56,79.06
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [20]:
#Note: there is no need to use Binning and pd.cut() in this instance. 
#It is more elegant/efficient to use the School Type column to groupby.

#Set School Type as index
school_type_index_df = per_school_summary.set_index('School Type')

#Groupby School Type and relevant columns
st_maths = school_type_index_df.groupby(['School Type'])['Average Maths Score'].mean().round(2)
st_reading = school_type_index_df.groupby(['School Type'])['Average Reading Score'].mean().round(2)
st_p_maths = school_type_index_df.groupby(['School Type'])['% Passing Maths'].mean().round(2)
st_p_reading = school_type_index_df.groupby(['School Type'])['% Passing Reading'].mean().round(2)
st_p_overall = school_type_index_df.groupby(['School Type'])['% Overall Passing'].mean().round(2)

#Create new DataFrame for above data
type_summary = pd.DataFrame({"Average Maths Score": st_maths,
                            "Average Reading Score": st_reading,
                            "% Passing Maths": st_p_maths,
                            "% Passing Reading": st_p_reading,
                            "% Overall Passing": st_p_overall})
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.97
